In [ ]:
import tensorflow as tf
import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam
import cv2
import os


In [ ]:
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import label_binarize
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, roc_auc_score, auc
from sklearn.multiclass import OneVsRestClassifier

In [ ]:
dataset_path = 'Datasets/lower modifier'

In [ ]:
image_size = (32,32)
batch_size = 10

#Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='training'
)

validation_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='validation'
)

test_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='validation',
    shuffle=False
)

In [ ]:
train_batches.class_indices

In [ ]:
l_characters = ['ु','ू','्','ृ']

In [ ]:
def plotImages(image_arr):
    fig, axes = plt.subplots(1,5)
    axes = axes.flatten()
    for img, ax in zip(image_arr, axes):
        ax.imshow(img[:,:,0])
        ax.axis('off')
    plt.tight_layout()
    plt.show()

imgs, labels = train_batches[0]
plotImages(imgs)
for i in range(5):
  print(l_characters[int(labels[:5][i])],end='\t')

In [ ]:
imgs.shape

In [ ]:
test_batches.classes

In [ ]:
labels.shape

In [ ]:
model1 = Sequential()

model1.add(Conv2D(128, (3, 3), strides = 1, activation = 'relu', input_shape = (32, 32, 1)))
model1.add(MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3, 3), strides = 1, activation = 'relu'))
model1.add(MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
model1.add(Dropout(0.25))

model1.add(Conv2D(128, (3, 3), strides = 1, activation = 'relu'))
model1.add(MaxPooling2D((2, 2), strides = (2, 2), padding = 'same'))
model1.add(Dropout(0.25))

model1.add(Flatten())

model1.add(Dense(128, activation = 'relu', kernel_initializer = 'he_uniform'))
model1.add(Dropout(0.2))

model1.add(Dense(50, activation = 'relu', kernel_initializer = 'he_uniform'))
model1.add(Dropout(0.2))

model1.add(Dense(4, activation = 'softmax'))

In [ ]:
model1.summary()

In [ ]:
model1.compile(optimizer = Adam(learning_rate = 1e-3, decay = 1e-5), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
if not os.path.isdir('Models/l_Model_1'):
    os.mkdir('Models/l_Model_1')

In [ ]:
callbacks1 = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1,
                              patience = 7, min_lr = 1e-5),
             EarlyStopping(monitor = 'val_loss', patience = 9, # Patience should be larger than the one in ReduceLROnPlateau
                          min_delta = 1e-5),
             CSVLogger(os.path.join('Models/l_Model_1','l_training_1.csv'), separator =',', append = True),
             ModelCheckpoint(os.path.join('Models/l_Model_1', 'backup_last_model.hdf5')),
             ModelCheckpoint(os.path.join('Models/l_Model_1', 'best_val_acc.hdf5'), monitor = 'val_accuracy', mode = 'max', save_best_only = True),
             ModelCheckpoint(os.path.join('Models/l_Model_1', 'best_val_loss.hdf5'), monitor = 'val_loss', mode = 'min', save_best_only = True)]

In [ ]:
model1.fit(train_batches, epochs = 20, validation_data = validation_batches, callbacks = callbacks1)

In [ ]:
model1 = load_model(os.path.join('Models/l_Model_1', 'best_val_loss.hdf5'))

In [ ]:
i=0
pred1=[]
num1=model1.predict(test_batches)
l=len(num1)
print(num1)
for i in range(l):
    val=np.argmax(num1[i])
    i+=1
    pred1.append(val)

In [ ]:
l_cm1=confusion_matrix(test_batches.classes,pred1)
print(l_cm1)

In [ ]:
print(accuracy_score(test_batches.classes,pred1))
#print(precision_score(test_batches.classes,pred1))
#print(recall_score(test_batches.classes,pred1))
#print(f1_score(test_batches.classes,pred1))

In [ ]:
nor_l_cm1=(np.round(l_cm1/np.sum(l_cm1,axis=1).reshape(-1,1),4))
print(nor_l_cm1)

In [ ]:
l_labels=['Rasho uoo','Dirga uoo','Khutta taneko','Re']

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['font.size'] = 15
plt.rcParams['savefig.facecolor']='white'
display_c_m = ConfusionMatrixDisplay(l_cm1, display_labels=l_labels)
display_c_m.plot(cmap='OrRd', xticks_rotation=25)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Confusion Matrix-1 for lower Modifier', fontsize=18)
plt.savefig('l_confusion_matrix1.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['font.size'] = 15
display_c_m = ConfusionMatrixDisplay(nor_l_cm1, display_labels=l_labels)
display_c_m.plot(cmap='OrRd', xticks_rotation=25)
plt.rcParams['savefig.facecolor']='white'
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Normalized Confusion Matrix-1 for lower Modifier', fontsize=18)
plt.savefig('Nor_l_confusion_matrix1.png', bbox_inches='tight')
plt.show()

In [ ]:
print(classification_report(test_batches.classes,pred1))

In [ ]:
roc_auc_score(test_batches.classes,num1, multi_class="ovr")

In [ ]:
test_batches.classes

In [ ]:
from sklearn.preprocessing import LabelBinarizer

label_binarizer = LabelBinarizer().fit(train_batches)
y_onehot_test = label_binarizer.transform(test_batches)
y_onehot_test.shape  # (n_samples, n_classes)

In [ ]:
test_binarized = label_binarize(test_batches, classes=[0, 1, 2, 3])

fpr={}
tpr={}
thresh={}
roc_auc = dict()

n_class = num1.shape[0]
for i in range(n_class):
    fpr[i], tpr[i], thresh[i] = roc_curve(test_binarized[:,i], num1[:,i])
    roc_auc[i] = auc(fpr[i], tpr[i])

    plt.plot(fpr[i], tpr[i], linestyle='--',
             label='%s vs rest'%(test_batches.class_indices[i],roc_auc[i]))

plt.plot([0,1],[0,1],'b--')
plt.xlim([0,1])
plt.ylim([0,1.05])
plt.title('ROC curve')
plt.xlabel('False positive rate')
plt.ylabel('True Positive rate')
plt.legend(loc='lower right')
plt.show()

In [ ]:
#fpr, tpr, thresholds = roc_curve(test_batches.classes, num1, pos_label = 1)

In [ ]:
model2 = Sequential()

model2.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform', input_shape = (32, 32, 1)))
model2.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Conv2D(32, (5, 5), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(MaxPooling2D((2, 2), strides = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Conv2D(64, (3, 3), padding = 'same', activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(MaxPooling2D((2, 2), strides = (2, 2)))
model2.add(Dropout(0.25))

model2.add(Flatten())

model2.add(Dense(256, activation = 'relu', kernel_initializer = 'he_uniform'))
model2.add(Dropout(0.25))

model2.add(Dense(4, activation = 'softmax'))

In [ ]:
model2.summary()

In [ ]:
if not os.path.isdir('Models/l_Model_2'):
    os.mkdir('Models/l_Model_2')

In [ ]:
callbacks2 = [ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1,
                              patience = 7, min_lr = 1e-5),
             EarlyStopping(monitor = 'val_loss', patience = 9, # Patience should be larger than the one in ReduceLROnPlateau
                          min_delta = 1e-5),
             CSVLogger(os.path.join('Models/l_Model_2','l_training_2.csv'), separator =',', append = True),
             ModelCheckpoint(os.path.join('Models/l_Model_2', 'backup_last_model.hdf5')),
             ModelCheckpoint(os.path.join('Models/l_Model_2', 'best_val_acc.hdf5'), monitor = 'val_accuracy', mode = 'max', save_best_only = True),
             ModelCheckpoint(os.path.join('Models/l_Model_2', 'best_val_loss.hdf5'), monitor = 'val_loss', mode = 'min', save_best_only = True)]

In [ ]:
model2.compile(optimizer = Adam(learning_rate = 1e-3, decay = 1e-5), 
               loss = 'sparse_categorical_crossentropy', 
               metrics = ['accuracy'])

In [ ]:
model2.fit(train_batches, epochs = 20, validation_data = validation_batches, callbacks = callbacks2)

In [ ]:
model2 = load_model(os.path.join('Models/l_Model_2', 'best_val_loss.hdf5'))

In [ ]:
i=0
pred2=[]
num2=model2.predict(test_batches)
l=len(num2)
print(num2)
for i in range(l):
    val=np.argmax(num2[i])
    i+=1
    pred2.append(val)

#print(pred2)

In [ ]:
l_cm2=confusion_matrix(test_batches.classes,pred2)
print(l_cm2)

In [ ]:
nor_l_cm2=(np.round(l_cm2/np.sum(l_cm2,axis=1).reshape(-1,1),4))
print(nor_l_cm1)

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['font.size'] = 15
plt.rcParams['savefig.facecolor']='white'
display_c_m = ConfusionMatrixDisplay(l_cm2, display_labels=l_labels)
display_c_m.plot(cmap='OrRd', xticks_rotation=25)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Confusion Matrix-2 for lower Modifier', fontsize=18)
plt.savefig('l_confusion_matrix2.png', bbox_inches='tight')
plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 10.0)
plt.rcParams['font.size'] = 15
plt.rcParams['savefig.facecolor']='white'
display_c_m = ConfusionMatrixDisplay(nor_l_cm2, display_labels=l_labels)
display_c_m.plot(cmap='OrRd', xticks_rotation=25)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title('Normalized Confusion Matrix-2 for lower Modifier', fontsize=18)
plt.savefig('Nor_l_confusion_matrix2.png', bbox_inches='tight')
plt.show()

In [ ]:
print(classification_report(test_batches.classes,pred2))